In [14]:
import joblib

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import params

np.random.seed(0)

### Define low-level DCM "LL" and high-level DCM "HL"

In [2]:
experiment = 'synth1'

In [3]:
S = 'Smoking'
T = 'Tar'
C = 'Cancer'

S_ = 'Smoking_'
C_ = 'Cancer_'

In [4]:
ll_endogenous_coeff_dict = {(S, T): 0.3, (T, C): 0.2}
ll_causal_graph          = CBN(list(ll_endogenous_coeff_dict.keys()))
#nx.draw(nx.DiGraph(ll_causal_graph.edges()),with_labels=True)

hl_endogenous_coeff_dict = {(S_, C_): 0.6}
hl_causal_graph          = CBN(list(hl_endogenous_coeff_dict.keys()))
#nx.draw(nx.DiGraph(hl_causal_graph.edges()),with_labels=True)

In [5]:
# Define the radius of the Wasserstein balls (epsilon, delta) and the size for both models.
epsilon         = params.radius[experiment][0]
ll_num_envs     = params.n_envs[experiment][0]

delta           = params.radius[experiment][1]
hl_num_envs     = params.n_envs[experiment][1]

# Define the number of samples per environment. Currently every environment has the same number of samples
num_llsamples   = params.n_samples[experiment][0]
num_hlsamples   = params.n_samples[experiment][1]

### Construct the empirical nominal distribution/ environment

In [6]:
ll_mu_hat    = np.array([0, 0, 0])  
ll_Sigma_hat = np.diag([1, 2, 1]) 

hl_mu_hat    = np.array([0, 0])  
hl_Sigma_hat = np.diag([1, 1])


ll_moments   = [(ll_mu_hat, ll_Sigma_hat)]
hl_moments   = [(hl_mu_hat, hl_Sigma_hat)]


lenv         = mut.sample_distros_Gelbrich(ll_moments)[0] 
henv         = mut.sample_distros_Gelbrich(hl_moments)[0]

In [7]:
Dll_noise      = lenv.sample(num_llsamples)[0]
ll_environment = mut.get_exogenous_distribution(Dll_noise)

Dhl_noise      = henv.sample(num_hlsamples)[0]
hl_environment = mut.get_exogenous_distribution(Dhl_noise)

/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


### Define the sets of relevant interventions and the (total) surjective and order-preserving function $ω:I^{L} \mapsto I^{H}$

In [8]:
iota0 = None
iota1 = ops.Intervention({S:0})
iota2 = ops.Intervention({S:0, T:1})
iota3 = ops.Intervention({S:1})
iota4 = ops.Intervention({S:1, T:0})
iota5 = ops.Intervention({S:1, T:1})

eta0 = None
eta1 = ops.Intervention({S_:0})
eta2 = ops.Intervention({S_:1})

omega = {   
            iota0: eta0,
            iota1: eta1,
            iota2: eta1,
            iota3: eta2,
            iota4: eta2,
            iota5: eta2
        }

Ill_relevant = list(set(omega.keys()))
Ihl_relevant = list(set(omega.values()))

In [9]:
joblib.dump((ll_causal_graph, Ill_relevant), f"data/{experiment}/LL.pkl")

joblib.dump((hl_causal_graph, Ihl_relevant), f"data/{experiment}/HL.pkl")

joblib.dump(omega, f"data/{experiment}/omega.pkl")

['data/synth1/omega.pkl']

### Sampling and Pair construction

In [10]:
Dll_samples = {}
for iota in Ill_relevant:
    llcm              = lanm.LinearAddSCM(ll_causal_graph, ll_endogenous_coeff_dict, iota)
    Dll_samples[iota] = llcm.sample_settings(Dll_noise)

Dhl_samples = {}
for eta in Ihl_relevant:
    hlcm             = lanm.LinearAddSCM(hl_causal_graph, hl_endogenous_coeff_dict, eta)
    Dhl_samples[eta] = hlcm.sample_settings(Dhl_noise)

Ds = {}
for iota in Ill_relevant:
    Ds[iota] = (Dll_samples[iota], Dhl_samples[omega[iota]])

In [11]:
joblib.dump(Ds, f"data/{experiment}/Ds.pkl")

['data/synth1/Ds.pkl']

In [12]:
# LLmodels, Dll_samples = {}, {}
# for iota in Ill_relevant:

#     LLmodels[iota]      = lanm.LinearAddSCM(ll_causal_graph, ll_endogenous_coeff_dict, iota)
#     Dll_samples[iota]   = LLmodels[iota].sample_settings(Dll_noise)

# HLmodels, Dhl_samples = {}, {}
# for eta in Ihl_relevant:

#     HLmodels[eta]      = lanm.LinearAddSCM(hl_causal_graph, hl_endogenous_coeff_dict, eta)
#     Dhl_samples[eta]   = HLmodels[eta].sample_settings(Dhl_noise)


# pairs = mut.create_pairs(Ill_relevant, omega, LLmodels, HLmodels)

# Ds = {}
# for iota in Ill_relevant:
#     Ds[iota] = (Dll_samples[iota], Dhl_samples[omega[iota]])

In [13]:
# joblib.dump(pairs, f"data/{experiment}/pairs.pkl")
# joblib.dump(Ds, f"data/{experiment}/Ds.pkl")